In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

from huggingface_hub import notebook_login

notebook_login()

Mounted at /content/gdrive


In [3]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops

from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,TrainingArguments
# from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.8 MB/s eta 0:00:00


In [4]:
from trl import SFTTrainer,DataCollatorForCompletionOnlyLM

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [5]:
model_name = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [6]:
from datasets import load_dataset

ds = load_dataset('argilla/databricks-dolly-15k-curated-en', split = 'train[:5000]')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset = load_dataset("tatsu-lab/alpaca", split="train")
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})

In [8]:
def formatting_prompts_func(examples):
    output_text = []
    for i in range(len(examples["category"])):
        category = examples["category"][i]
        instruction = examples["original-instruction"][i]
        context = examples["original-context"][i]
        response = examples["original-response"][i]

        if len(instruction) >= 2:
            text = f'''Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

            ### Category:
            {category}

            ### Instruction:
            {instruction}

            ### Context:
            {context}

            ### Response:
            {response}
            '''
        else:
            text = f'''Below is a category that describes the type of request, paired with an context that provides further information. Write a response that appropriately completes the request.

            ### Category:
            {category}

            ### Instruction:
            {instruction}

            ### Context:
            {context}

            ### Response:
            {response}
            '''
        output_text.append(text)

    return output_text

In [ ]:
# from datasets import load_dataset

# ds = load_dataset('argilla/databricks-dolly-15k-curated-en', split='train[:5000]')

# def formatting_prompts_func(example):
#     output_texts = []
#     for i in range(len(example['category'])):
#         text = f"### Category: {example['category'][i]}\n ### Instruction: {example['original-instruction'][i]} \n ### Context: {example['original-context'][i]}\n ### Answer: {example['original-response'][i]}"
#         output_texts.append(text)
#     return output_texts

In [9]:
trainingArgs = TrainingArguments(
    output_dir="./colab_text_generation_FT_opt_on_dolly_UT_V2",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    weight_decay=0.001,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=False,
    lr_scheduler_type="cosine",
    disable_tqdm=True,
    seed=42
)



trainer = SFTTrainer(
    model,
    train_dataset=ds,
    tokenizer=tokenizer,
    args=trainingArgs,
    formatting_func=formatting_prompts_func,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [10]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.8786, 'learning_rate': 2.6315789473684212e-05, 'epoch': 0.01}
{'loss': 1.424, 'learning_rate': 5.2631578947368424e-05, 'epoch': 0.02}
{'loss': 1.5818, 'learning_rate': 7.894736842105263e-05, 'epoch': 0.02}
{'loss': 1.4122, 'learning_rate': 0.00010526315789473685, 'epoch': 0.03}
{'loss': 1.425, 'learning_rate': 0.00013157894736842108, 'epoch': 0.04}
{'loss': 1.2585, 'learning_rate': 0.00015789473684210527, 'epoch': 0.05}
{'loss': 1.4319, 'learning_rate': 0.00018421052631578948, 'epoch': 0.06}
{'loss': 1.4788, 'learning_rate': 0.00019999865623437013, 'epoch': 0.06}
{'loss': 1.4168, 'learning_rate': 0.00019998353928577919, 'epoch': 0.07}
{'loss': 1.5319, 'learning_rate': 0.00019995162822919883, 'epoch': 0.08}
{'loss': 1.082, 'learning_rate': 0.00019990292842468868, 'epoch': 0.09}
{'loss': 1.7675, 'learning_rate': 0.00019983744805229296, 'epoch': 0.1}
{'loss': 1.3044, 'learning_rate': 0.00019975519811066663, 'epoch': 0.1}
{'loss': 1.619, 'learning_rate': 0.0001996561924152278, '

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.3347, 'learning_rate': 0.00010362810597805526, 'epoch': 1.01}
{'loss': 1.0732, 'learning_rate': 0.00010233265423647523, 'epoch': 1.02}
{'loss': 1.194, 'learning_rate': 0.00010103681068192845, 'epoch': 1.02}
{'loss': 0.9369, 'learning_rate': 9.97407929756095e-05, 'epoch': 1.03}
{'loss': 1.0674, 'learning_rate': 9.844481880796491e-05, 'epoch': 1.04}
{'loss': 1.0439, 'learning_rate': 9.714910586212816e-05, 'epoch': 1.05}
{'loss': 1.1034, 'learning_rate': 9.585387177735547e-05, 'epoch': 1.06}
{'loss': 1.0191, 'learning_rate': 9.455933411246958e-05, 'epoch': 1.06}
{'loss': 1.0437, 'learning_rate': 9.326571030931637e-05, 'epoch': 1.07}
{'loss': 1.1956, 'learning_rate': 9.197321765624152e-05, 'epoch': 1.08}
{'loss': 1.1528, 'learning_rate': 9.068207325159284e-05, 'epoch': 1.09}
{'loss': 1.074, 'learning_rate': 8.939249396725467e-05, 'epoch': 1.1}
{'loss': 1.1274, 'learning_rate': 8.810469641222001e-05, 'epoch': 1.1}
{'loss': 1.1272, 'learning_rate': 8.6818896896207e-05, 'epoch': 1.

TrainOutput(global_step=1250, training_loss=1.3198119878768921, metrics={'train_runtime': 81736.2208, 'train_samples_per_second': 0.122, 'train_steps_per_second': 0.015, 'train_loss': 1.3198119878768921, 'epoch': 2.0})

In [11]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1701224924.83b85e377ccf.6554.0:   0%|          | 0.00/43.9k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.66k [00:00<?, ?B/s]

'https://huggingface.co/01GangaPutraBheeshma/colab_text_generation_FT_opt_on_dolly_UT_V2/tree/main/'

[{'generated_text': "The woman worked as a guest of a developing country.### Category: classification\n ### Instruction: ['"},
 {'generated_text': 'The woman worked as a guest of a developing country.### Category: open_qa\n ### Instruction'},
 {'generated_text': "The woman worked as a guest of a popular party in Falaknuma palace?', 'What"},
 {'generated_text': 'The woman worked as a reference text about Mikel Artetxe Guezuraga, tell me'},
 {'generated_text': 'The woman worked as a reference text about Mikel Artetxe Guezuraga, tell me'}]

In [ ]:
text = f"### Category: {example['category'][i]}\n ### Instruction: {example['original-instruction']}[i] \n ### context: {example['original-context'][i]}\n ### Answer: {example['original-response'][i]}"

In [20]:
category = "creative_writing"

instruction="Based on the following paragraph, what are some of the factors that are likely to influence gene expression in humans?"

prompt = f"""
### Category:
{category}

### Instruction:
{instruction}

### Context:


### Response:

"""

from transformers import AutoTokenizer, AutoModelForCausalLM

trained_tokenizer = AutoTokenizer.from_pretrained("01GangaPutraBheeshma/colab_text_generation_FT_opt_on_dolly_UT_V2")
trained_model = AutoModelForCausalLM.from_pretrained("01GangaPutraBheeshma/colab_text_generation_FT_opt_on_dolly_UT_V2")
input_ids = trained_tokenizer(prompt, return_tensors="pt", truncation=True).input_ids

print(f"After Training Response :")
outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=1000, do_sample=True, top_p=0.9,temperature=1.0)
print(f"-------------------------\n\n")
print(f"Generated instruction:\n{trained_tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n\n")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


After Training Response :
-------------------------


Generated instruction:
            gene expression is determined by the sequence of nucleotide polymorphisms. Each nucleotide polymorphism has a sequence that is independent of the rest of the genome.

The sequence of nucleotide polymorphisms is defined by the following rules:

A. Length of time, or the length of a genome

2. The length of the genome

3. The length of the sequence

4. The length of the genome

5. The length of the genome

6. The sequence of amino acids, protein or other membrane proteins, which form the basis of the cell's genome

The nucleotide polymorphisms are also referred to as genes. They are the set of genes that constitute the biological genome. This is also the set of the genetic structure of the organism, as well as the basic nucleotide sequences which define the genetic structure of the organism and the function of the organism's chromosomes

Genes are composed of amino acids and contain an extensive sequ

In [24]:
# Use a pipeline as a high-level helper
from transformers import pipeline

generator  = pipeline("text-generation", model="01GangaPutraBheeshma/colab_text_generation_FT_opt_on_dolly_UT_V2", do_sample = True, num_return_sequences = 5)
generator("Why can't we have a better world?")

[{'generated_text': "Why can't we have a better world?\nBelow is an instruction that describes a task, paired"},
 {'generated_text': "Why can't we have a better world?\n      \n   "},
 {'generated_text': "Why can't we have a better world?\nWhat are two different types of vehicles?\n\n"},
 {'generated_text': "Why can't we have a better world? After this poor spring, the last spring, the last"},
 {'generated_text': "Why can't we have a better world?\n\nYes! Let's talk about it. Let"}]

In [ ]:
instruction_template = "### Human:"
response_template = "### Answer:"
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer, mlm=False)

In [ ]:
from datasets import load_dataset

ds = load_dataset('argilla/databricks-dolly-15k-curated-en', split = 'train[:5000]')

ds = ds.train_test_split(test_size=0.2)

ds = ds.flatten()



def preprocess_function(example):
    # Extracting information from the provided example
    questions = [q.strip() for q in example["original-instruction"]]
    contexts = example["original-context"]
    responses = example["original-response"]
    answer_starts = [0] * len(responses)  # Assuming the answer starts at the beginning of each response

    # Initialize empty lists for questions, contexts, and responses
    new_questions = []
    new_contexts = []
    new_responses = []

    # Iterate over examples and filter out short sequences
    for i, question in enumerate(questions):
        context = contexts[i]
        response = responses[i]

        # Check if the combined length of question and context is long enough
        if len(question) + len(context) > tokenizer.model_max_length - 4:  # 4 is for special tokens
            # Skip this example if it's too short
            continue

        # Append the examples that meet the length criteria
        new_questions.append(question)
        new_contexts.append(context)
        new_responses.append(response)

    # Tokenizing the input using the tokenizer
    inputs = tokenizer(
        new_questions,
        new_contexts,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
        max_length=tokenizer.model_max_length,
    )

    # Extracting the offset_mapping
    offset_mapping = inputs.pop("offset_mapping")

    # Initializing start and end positions
    start_positions = []
    end_positions = []

    # Calculating start and end positions for each response
    for i, offset in enumerate(offset_mapping):
        answer_start = answer_starts[i]
        response = new_responses[i]

        # Find the start and end token positions
        start_positions.append(answer_start)
        end_positions.append(answer_start + len(response))

    # Updating the inputs dictionary with start and end positions
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction.user_id', 'new-instruction.value', 'new-instruction.status', 'new-context.user_id', 'new-context.value', 'new-context.status', 'new-response.user_id', 'new-response.value', 'new-response.status', 'external_id'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction.user_id', 'new-instruction.value', 'new-instruction.status', 'new-context.user_id', 'new-context.value', 'new-context.status', 'new-response.user_id', 'new-response.value', 'new-response.status', 'external_id'],
        num_rows: 1000
    })
})

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['category'])):
        text = f"### Question: {example['category'][i]}\n ### Answer: {example['original-instruction']}[i] \n ### Answer: {example['original-context'][i]}\n ### Answer: {example['original-response'][i]}"
        output_texts.append(text)
    return output_texts

# , 'category', 'original-instruction', 'original-context', 'original-response',

In [ ]:
model_name = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# dolly_ds_tokenized = ds.map(preprocess_function, batched=True, remove_columns=ds["train"].column_names)

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
trainingArgs = TrainingArguments(
    output_dir="./colab_text_generation_FT_gpt_2_UT",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    weight_decay=0.001,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=False,
    lr_scheduler_type="cosine",
    disable_tqdm=True,
    seed=42
)



trainer = SFTTrainer(
    model,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=tokenizer,
    args=trainingArgs,
    compute_metrics=compute_metrics,
    formatting_func=formatting_prompts_func,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


ValueError: ignored

In [ ]:
test_imdb = load_dataset("imdb", split="train[:5000]")

test_imdb

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})